# Kaggle Environment Setup

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("API")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

In [ ]:
import json
import requests
import subprocess
import time
import uuid

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# Making Model

In [ ]:
import pandas as pd
df = pd.read_csv("/kaggle/input/liar-dataset/train.tsv", sep='\t', header=None)

In [ ]:
df.columns = [
    "label", "statement", "subject", "speake", "job_title",
    "state_info", "party_affiliation", "barely_true_counts",
    "false_counts", "half_true_counts", "mostly_true_counts",
    "pants_on_fire_counts", "context","data"
]

df.head()

In [ ]:
texts = df['statement'].tolist()
labels = df['label'].map({'true':1, 'false':0}).tolist()

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encodings = tokenizer(texts, truncation=True, padding=True)

# Dataset class
class ClaimDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

dataset = ClaimDataset(encodings, labels)

#Model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Training
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()


In [ ]:
# !pip install transformers sentence-transformers datasets evaluate accelerate --quiet
from typing import List, Dict, Any
from dataclasses import dataclass

from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments
)
from sentence_transformers import SentenceTransformer, util

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Global config
CONFIG = {
    "retriever_model": "gemini-2.5-flash-lite",
    "verifier_model": "roberta-base",     # replace with your fine-tuned checkpoint later
    "top_k_evidence": 5,
    "memory_dir": "./radar_memory",
    "log_file": "./radar_logs.jsonl"
}

os.makedirs(CONFIG["memory_dir"], exist_ok=True)

# Helpers: logging + memory
def log_event(agent: str, stage: str, payload: Dict[str, Any]):
    event = {
        "ts": time.strftime("%Y-%m-%d %H:%M:%S"),
        "agent": agent,
        "stage": stage,
        "payload": payload
    }
    with open(CONFIG["log_file"], "a") as f:
        f.write(json.dumps(event) + "\n")

def memory_write(key: str, value: Dict[str, Any]):
    path = os.path.join(CONFIG["memory_dir"], f"{key}.json")
    with open(path, "w") as f:
        json.dump(value, f)

def memory_read(key: str):
    path = os.path.join(CONFIG["memory_dir"], f"{key}.json")
    return json.load(open(path)) if os.path.exists(path) else None


In [ ]:
def extract_claim(text: str) -> str:
    """
    Minimal baseline: trims and returns the main sentence.
    Extend with NER, dependency parsing, or heuristic rules to isolate factual propositions.
    """
    claim = text.strip()
    log_event("ClaimExtractor", "extracted", {"input": text, "claim": claim})
    return claim
    

corpus_df = pd.DataFrame({
    "source": ["Wikipedia", "Snopes", "WHO"],
    "snippet": [
        "Vaccines undergo rigorous testing for safety and efficacy.",
        "Claims about microchips in vaccines have been repeatedly debunked.",
        "WHO states that approved COVID-19 vaccines are safe and effective."
    ]
})

CORPUS = corpus_df["snippet"].tolist()

retriever = LlmAgent(
    name="retriever",  # required field
    instruction="\n".join(CORPUS),  # must be a string, not DataFrame
    model=Gemini(model=CONFIG["retriever_model"],
                tools=[extract_claim])  # use the correct field name
)
print("✅ Here is fact checker Agent created successfully!")
print("   Model: gemini-2.5-flash-lite")
print("   Tool: extract_claim()")
print("   Ready to be exposed via A2A...")

In [ ]:
# Test the agent
from google.adk.runners import InMemoryRunner
current_fact = InMemoryRunner(agent=retriever)
_ = await current_fact.run_debug(
    "Is it true, that COVID have make a sheid it self? How it make it?"
)